# Parallel XEB

In [ ]:
import cirq
import numpy as np

## Set up Random Circuits

We create a set of 20 random, two-qubit `circuits`

In [ ]:
from cirq.experiments import random_quantum_circuit_generation as rqcg

circuits = rqcg.generate_library_of_2q_circuits(
    n_library_circuits=20, 
    two_qubit_gate=cirq.ISWAP**0.5,
)
print(len(circuits))

In [ ]:
# We will truncate to these lengths
cycle_depths = np.arange(3, 75, 11)
cycle_depths

## Make a device to run in parallel on

In [ ]:
qubits = cirq.GridQubit.rect(3, 2, 4, 3)
qubits = qubits[:-1]  # make it irregular
qubits = qubits[:-2]  # make it tiny
sampler = cirq.DensityMatrixSimulator(noise=cirq.depolarize(5e-3))

import cirq.contrib.routing as ccr
import networkx as nx
graph = ccr.gridqubits_to_graph_device(qubits)
pos = {q: (q.row, q.col) for q in qubits}
nx.draw_networkx(graph, pos=pos)

In [ ]:
import cirq.contrib.routing as ccr
import cirq.google as cg
import os
os.environ['GOOGLE_CLOUD_PROJECT'] = 'google.com:quantum-experiments'
device_name = 'mcgee'
sampler = cg.get_engine_sampler(device_name, gate_set_name='sqrt_iswap')

device = cg.get_engine_device(device_name)
qubits = sorted(device.qubits)
if device_name == 'mcgee':
    qubits = qubits[:-2]
    qubits.remove(cirq.GridQubit(2,3))
if device_name == 'rainbow':
    qubits = cirq.GridQubit.rect(3,2,4,3)

graph = ccr.gridqubits_to_graph_device(qubits)
pos = {q: (q.row, q.col) for q in qubits}
nx.draw_networkx(graph, pos=pos)

## Set up our combinations
We take the library of two-qubit circuits and mix-and-match to run parallel efficiently.

We pass to the sampling function and it will "zip" the circuits according to these combinations which are given by a GridInteractionLayer (one of four for the degree-four GridQubit implied graph), and a row of circuit indices.

In [ ]:
combs_by_layer = rqcg.get_random_combinations_for_device(
    n_library_circuits=len(circuits),
    n_combinations=20,
    device_graph=graph,
)
combs_by_layer

In [ ]:
from matplotlib import pyplot as plt

fig, axes = plt.subplots(2,2, figsize=(9,6))
for comb_layer, ax in zip(combs_by_layer, axes.reshape(-1)):
    active_qubits = np.array(comb_layer.pairs).reshape(-1)
    colors = ['red' if q in active_qubits else 'blue' for q in graph.nodes]
    nx.draw_networkx(graph, pos=pos, node_color=colors, ax=ax)
    nx.draw_networkx_edges(graph, pos=pos, edgelist=comb_layer.pairs, width=3, edge_color='red', ax=ax)
    
plt.tight_layout()

In [ ]:
from cirq.experiments.fidelity_estimation import sample_2q_xeb_circuits
sampled_df = sample_2q_xeb_circuits(
    sampler=sampler,
    circuits=circuits,
    cycle_depths=cycle_depths,
    combinations_by_layer=combs_by_layer
)
sampled_df

In [ ]:
#sampled_df.to_pickle('02-03-mcgee.pickl')
#cirq.to_json(circuits, '02-03-mcgee.json')

In [ ]:
from cirq.experiments.fidelity_estimation import benchmark_2q_xeb_fidelities
# TODO: only simulate the circuits that are here
fids = benchmark_2q_xeb_fidelities(
    sampled_df=sampled_df,
    circuits=circuits,
    cycle_depths=cycle_depths,
)
fids

In [ ]:
# from cirq.experiments.fidelity_estimation import simulate_2q_xeb_circuits
# simulated_df = simulate_2q_xeb_circuits(circuits=circuits, cycle_depths=cycle_depths, param_resolver=None)
# simulated_df

In [ ]:
# df = sampled_df[sampled_df['pair_name'] == '(0, 6)-(1, 6)'].join(simulated_df)
# df

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

# Give each pair its own color
colors = sns.cubehelix_palette(n_colors=graph.number_of_edges())
colors = {f'{q0}-{q1}': color for (q0, q1), color in zip(graph.edges, colors)}

# Exponential reference
xx = np.linspace(0, fids['cycle_depth'].max())
plt.plot(xx, (1-5e-3)**(4*xx), label=r'Exponential Reference', color='black')

# Plot each pair
def _p(fids):
    plt.plot(fids['cycle_depth'], fids['fidelity'], 
             'o-', label=fids.name, color=colors[fids.name],
             alpha=0.5)
fids.groupby('pair_name').apply(_p)

plt.ylabel('Circuit fidelity')
plt.xlabel('Cycle Depth $d$')
plt.legend(loc='best')
plt.tight_layout()

## Optimize `PhasedFSimGate` parameters

We know what circuits we requested, and in this simulated example, we know what coherent error has happened. But in a real experiment, there is likely unknown coherent error that you would like to characterize. Therefore, we make the five angles in `PhasedFSimGate` free parameters and use a classical optimizer to find which set of parameters best describes the data we collected from the noisy simulator (or device, if this was a real experiment).

fids_opt = simulate_2q_xeb_fids(sampled_df, pcircuits, cycle_depths, param_resolver={'theta': -np.pi/4, 'phi': 0.1})

In [ ]:
import multiprocessing
pool = multiprocessing.get_context('spawn').Pool()

In [ ]:
sampled_df.set_index('pair_name').sort_index()

In [ ]:
from cirq.experiments.fidelity_estimation import \
    parameterize_phased_fsim_circuit, characterize_phased_fsim_parameters_with_xeb, SqrtISwapXEBOptions

options = SqrtISwapXEBOptions()
pcircuits = [parameterize_phased_fsim_circuit(circuit, options) for circuit in circuits]
res = characterize_phased_fsim_parameters_with_xeb(
                    sampled_df, pcircuits, cycle_depths, options, pool=pool, fatol=5e-3, xatol=5e-3)

In [ ]:
from cirq.experiments.fidelity_estimation import \
    parameterize_phased_fsim_circuit, characterize_phased_fsim_parameters_with_xeb_by_pair, SqrtISwapXEBOptions

options = SqrtISwapXEBOptions()
pcircuits = [parameterize_phased_fsim_circuit(circuit, options) for circuit in circuits]
res_by_pn = characterize_phased_fsim_parameters_with_xeb_by_pair(
                    sampled_df, pcircuits, cycle_depths, options, pool=pool, fatol=5e-3, xatol=5e-3)

In [ ]:
# TODO: put in the opty func.
opt_df_by_pn = {}
for pair_name in res_by_pn.keys():
    res = res_by_pn[pair_name]
    _, names = options.get_initial_simplex_and_names()
    final_params = dict(zip(names, res.x))
    print(final_params)
    
    fids_opt = benchmark_2q_xeb_fidelities(
        sampled_df[sampled_df['pair_name'] == pair_name], pcircuits, cycle_depths, 
        param_resolver=final_params)
    
    opt_df_by_pn[pair_name] = fids_opt

In [ ]:
# fids_opt = benchmark_2q_xeb_fidelities(
#     sampled_df[sampled_df['pair_name'] == pair_name], pcircuits, cycle_depths, 
#     param_resolver=final_params)

In [ ]:
import pandas as pd
fids_opt = pd.concat(opt_df_by_pn.values())
fids_opt

In [ ]:
xx = np.linspace(0, fids['cycle_depth'].max())
plt.plot(xx, (1-5e-3)**(4*xx), label=r'Exponential Reference', color='black')
plt.axhline(1, color='grey', ls='--')

#plt.plot(fids['cycle_depth'], fids['fidelity'], 'o-', label='Raw')
#plt.plot(fids_opt['cycle_depth'], fids_opt['fidelity'], 'o-', label='Refit')

# Plot each pair
def _p(fids, marker='.', ls='-'):
    plt.plot(fids['cycle_depth'], fids['fidelity'], 
             marker=marker, ls=ls, label=fids.name, color=colors[fids.name],
             alpha=0.9)

fids.groupby('pair_name').apply(_p, ls='--', marker='')
fids_opt.groupby('pair_name').apply(_p)
#plt.yscale('log')

plt.ylabel('Circuit fidelity')
plt.xlabel('Cycle Depth')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
import scipy.optimize

def exp(x, A, fid):
    # TODO: is the 4 appropriate here?
    #return A * fid**(4*x) + b
    return A * np.exp(np.log(fid) * x)
    

xx = np.linspace(0, fids['cycle_depth'].max())
for pair_name in fids['pair_name'].unique():
    f1 = fids[fids['pair_name'] == pair_name]
    f2 = fids_opt[fids_opt['pair_name'] == pair_name]
    
    #plt.plot(xx, (1-5e-3)**(4*xx), label=r'Exponential Reference', color='black')
    plt.axhline(1, color='grey', ls='--')
    plt.plot(f1['cycle_depth'], f1['fidelity'], '.--')
    plt.plot(f2['cycle_depth'], f2['fidelity'], 'o-')
    
    popt, pcov = scipy.optimize.curve_fit(exp, f2['cycle_depth'], f2['fidelity'], p0=(0.95, 5e-3),
                                         bounds=(
                                             [0, 0,],
                                             [1, 1,],
                                         ))
    print(popt)
    plt.plot(xx, exp(xx, *popt), label='Fit')
    
    plt.ylabel('Circuit fidelity')
    plt.xlabel('Cycle Depth')
    plt.tight_layout()
    plt.show()